#Libraries & SetUp

In [ ]:
%pip install -q -U langchain-huggingface langchain-community langgraph langchain-groq PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 25.9 MB/s eta 0:00:00


In [ ]:
import os, sys, json, unicodedata, random
from openai import OpenAI

from langchain.chat_models import init_chat_model
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import CSVLoader
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langgraph.graph import START, StateGraph
from langchain_core.documents import Document
from langchain import hub
from langchain_core.output_parsers import JsonOutputParser

from typing_extensions import List, TypedDict
from pydantic import BaseModel, Field
from typing import List
import re, pandas as pd, numpy as np, time, PyPDF2

In [ ]:
# add api keys

In [ ]:
client = OpenAI(api_key = OPENAI_API_KEY)
chunking_strategy =  {
        "type": "static",
        "static": {
          "max_chunk_size_tokens": 800, # reduce size to ensure better context integrity
          "chunk_overlap_tokens": 400 # increase overlap to maintain context across chunks
        }}

# Create a vector store caled "Scientific articles"
vector_store = client.vector_stores.create(name="Scientific articles",
                                                chunking_strategy=chunking_strategy)

In [ ]:
import groq

# Initialize Groq client for listing models
# Replace with your actual API key
groq_client = groq.Client(api_key="YOUR_GROQ_API_KEY")

import groq
groq_client = groq.Client(api_key=GROQ_API_KEY)

models = groq_client.models.list()
print("Available models for Groq provider:")
for model in models.data:
  print(f" - {model.id}")


Available models for Groq provider:
 - compound-beta
 - llama-3.3-70b-versatile
 - whisper-large-v3-turbo
 - llama3-8b-8192
 - meta-llama/llama-prompt-guard-2-86m
 - qwen/qwen3-32b
 - meta-llama/llama-4-maverick-17b-128e-instruct
 - playai-tts
 - meta-llama/llama-guard-4-12b
 - allam-2-7b
 - playai-tts-arabic
 - llama-3.1-8b-instant
 - mistral-saba-24b
 - meta-llama/llama-prompt-guard-2-22m
 - llama3-70b-8192
 - gemma2-9b-it
 - compound-beta-mini
 - whisper-large-v3
 - moonshotai/kimi-k2-instruct
 - distil-whisper-large-v3-en
 - meta-llama/llama-4-scout-17b-16e-instruct
 - deepseek-r1-distill-llama-70b


In [ ]:
models = groq_client.models.list()
print("Available models for Groq provider (filtered for 'llama'):")
for model in models.data:
  if "llama" in model.id:
    print(f" - {model.id}")

Available models for Groq provider (filtered for 'llama'):
 - meta-llama/llama-prompt-guard-2-22m
 - meta-llama/llama-4-scout-17b-16e-instruct
 - llama-3.3-70b-versatile
 - meta-llama/llama-guard-4-12b
 - meta-llama/llama-prompt-guard-2-86m
 - llama3-8b-8192
 - deepseek-r1-distill-llama-70b
 - llama-3.1-8b-instant
 - meta-llama/llama-4-maverick-17b-128e-instruct
 - llama3-70b-8192


In [ ]:
# llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq", temperature = 0)
llm = init_chat_model("llama-3.1-8b-instant", model_provider="groq", temperature = 0)

#System check on a small text

In [ ]:
# from PyPDF2 import PdfReader

# path = "/content/Black_Cat.pdf"
# reader = PdfReader(path)

# # Extract all the text into one string
# all_text = "\n".join(page.extract_text() or "" for page in reader.pages)

# # Build your messages: one system prompt + one user prompt
# messages_simple = [
#     {
#         "role": "system",
#         "content": (
#             "Give the name of the attached pdf and its author in json table with one column - name"
#             """Return the output as a **stringified JSON array**, with no extra text:
#         [
#             {{
#             "name": "<answer>",
#             "author": "<answer>"
#             }}
#         ]
# """
#         )
#     },
#     {
#         "role": "user",
#         "content": all_text
#     }
# ]

# result = llm.invoke(input=messages_simple)
# # Groq llama wrapper returns an AIMessage; extract text via .content
# assistant_response = result.content if hasattr(result, 'content') else str(result)
# print(assistant_response)

#JSON extraction

In [ ]:
def extract_json(text: str) -> object:
    pattern = r"(?P<json>\[.*?\]|\{.*?\})"
    match = re.search(pattern, text, flags=re.DOTALL)
    if not match:
        raise ValueError("No JSON block found in the input text.")

    json_str = match.group("json")
    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"Failed to parse JSON: {e}\nExtracted string: {json_str}")


def json_to_dataframe(json_data: object) -> pd.DataFrame:
    if isinstance(json_data, dict):
        df = pd.DataFrame([json_data])
    elif isinstance(json_data, list):
        df = pd.DataFrame(json_data)
    else:
        raise ValueError("JSON data is neither a dict nor a list of dicts.")
    return df

In [ ]:
# data = extract_json(assistant_response)
# df = json_to_dataframe(data)
# print(f"\n--- Response ---\n")
# print(df)

#Data installation & preprocessing

In [ ]:
!gdown --folder "https://drive.google.com/drive/u/0/folders/1-66VrFbR6OgwNiNcCoOZ0FNeLy7b5ITC" > /dev/null 2>&1
!gdown --remaining-ok --folder "https://drive.google.com/drive/u/0/folders/1zurB2MibVbrhiobBe5oyvxcK6n5hdbB_" > /dev/null 2>&1

In [ ]:
assistant_data = '/content/lab/files'

# print("Files to upload:")
file_paths = []
for root, dirs, files in os.walk(assistant_data):
    # Filter out hidden dirs
    dirs[:] = [d for d in dirs if not d.startswith('.')]
    for file in files:
        # Skip hidden and notebook/config files
        if file.startswith('.') or file.endswith(('.ipynb', '.yaml', '.txt')):
            continue
        # print(f"\t- {file}")
        file_paths.append(os.path.join(root, file))

print(f"\nTotal files found: {len(file_paths)}")



def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])

def format(study_name_string):
    """
    Cuts a string after the last four-digit number, assuming it represents the year.

    Args:
        study_name_string (str): The input string potentially containing a year.

    Returns:
        str: The string cut after the year, or the original string if no year is found.
    """
    # Get rid of all the points, -
    study_name_string = study_name_string.replace('.', '')
    study_name_string = study_name_string.replace(',', '')
    study_name_string = study_name_string.replace(' - ', ' ')
    study_name_string = study_name_string.replace(')', '')
    study_name_string = study_name_string.replace('(', '')
    study_name_string = study_name_string.replace('&', 'and')
    study_name_string = remove_accents(study_name_string)
    # Find all occurrences of four consecutive digits (potential years)
    year_matches = list(re.finditer(r'\b\d{4}\b', study_name_string))

    if year_matches:
        # Get the last match
        last_year_match = year_matches[-1]
        # Get the end index of the last year match
        end_of_year_index = last_year_match.end()
        # Slice the string up to the end of the year
        cut_string = study_name_string[:end_of_year_index]
        return cut_string.strip() # Use strip to remove trailing whitespace
    else:
        # If no four-digit number is found, return the original string
        return study_name_string.strip()


# Read the Excel file
df = pd.read_excel('/content/table/data.xlsx')
df = df[['Study_ID', 'Study', 'Allocation', 'Experimenter']]

# Remove duplicates based on Study_ID and Study before grouping
df = df.drop_duplicates(subset=['Study_ID', 'Study']).reset_index(drop=True)


# Group by 'Study_ID' (case-sensitive)
output_dir = '/data/grouped_csv'
os.makedirs(output_dir, exist_ok=True)
print(f"Directory '{output_dir}' created successfully.\n")

grouped = df.groupby('Study_ID')

# Ensure the 'Study' column exists and is in a comparable format (string)
if 'Study' not in df.columns:
    print("Error: 'Study' column not found in the DataFrame.")
# Ensure 'Study' column is string type for consistent comparison
df['Study'] = df['Study'].astype(str)


study_to_group_map = {}
for study_id, group_df in grouped:
    if not group_df['Study'].empty:
        study_name_from_df = format(group_df['Study'].iloc[0])
        study_to_group_map[study_name_from_df] = group_df

matched_pairs = []

for file_path in file_paths:
    filename = os.path.basename(file_path)
    cleaned_filename_study_name = format(filename)

    matching_group_df = study_to_group_map.get(cleaned_filename_study_name)

    if matching_group_df is not None:
        # print(f"  - Matched file '{cleaned_filename_study_name}' with Study '{cleaned_filename_study_name}'")
        matched_pairs.append((file_path, matching_group_df))
    else:
        print(f"  - No matching DataFrame group found for cleaned filename '{cleaned_filename_study_name}' (from file '{filename}')")

print(f"\nFound {len(matched_pairs)} matched pairs.")


Total files found: 40
Directory '/data/grouped_csv' created successfully.

  - No matching DataFrame group found for cleaned filename 'Fielding‐Barnsley and Purdie 2003' (from file 'Fielding‐Barnsley and Purdie - 2003 - Early intervention in the home for children at risk of reading failure.pdf')
  - No matching DataFrame group found for cleaned filename '1html' (from file '1.html')
  - No matching DataFrame group found for cleaned filename 'Gonzalez et al 2010' (from file 'Gonzalez et al. - 2010 - Developing Low-Income Preschoolers’ Social Studies and Science Vocabulary Knowledge Through Content-.pdf')
  - No matching DataFrame group found for cleaned filename 'Kotaman IMPACTS OF DIALOGICAL STORYBOOK READING ON YOUNG CHILDREN'S READING ATTITUDES AND VOCABULARY DEVELOPpdf' (from file 'Kotaman - IMPACTS OF DIALOGICAL STORYBOOK READING ON YOUNG CHILDREN'S READING ATTITUDES AND VOCABULARY DEVELOP.pdf')
  - No matching DataFrame group found for cleaned filename 'Nelson et al 2009' (from fi

#Evaluation

In [ ]:
def accuracy_check(col_name, df_extracted, test_table):
  allocation_match = False
  experimenter_match = False


  # Check if both dataframes have the expected columns and rows
  if col_name in df_extracted.columns and \
    not df_extracted.empty and not test_table.empty:

      extracted_allocation = df_extracted[col_name].iloc[0]

      ground_truth_allocation = test_table[col_name].iloc[0]

      # Simple case-insensitive comparison
      if str(extracted_allocation).lower() == str(ground_truth_allocation).lower():
          allocation_match = True
          print(f'{col_name}: Match')
      else:
          print(f"{col_name}: Mismatch (Extracted: '{extracted_allocation}', Ground Truth: '{ground_truth_allocation}')")
  else:
    print("Cannot perform accuracy check: Extracted or ground truth data is missing or malformed.")
  print("--------------------")

In [ ]:
def calculateAccuracy(TP, FP, FN, TN):
    '''calculates the accuracy of a model based on true positives, true negatives, false positives, and false negatives.'''
    return (TP + TN) / (TP + FP + FN + TN) if (TP + FP + FN + TN) > 0 else 0

def calculatePrecision(TP, FP):
    '''calculates the precision of a model based on true positives and false positives.'''
    return TP / (TP + FP) if (TP + FP) > 0 else 0

def calculateRecall(TP, FN):
    '''calculates the recall of a model based on true positives and false negatives.'''
    return TP / (TP + FN) if (TP + FN) > 0 else 0

def calculateF1Score(precision, recall):
    '''calculates the F1 score based on precision and recall.'''
    return 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

def calculateMetrics(TP, FP, FN, TN):
    '''calculates various metrics based on true positives, false positives, and false negatives.'''
    # --- Metrics ---
    accuracy = calculateAccuracy(TP, FP, FN, TN)
    recall = calculateRecall(TP, FN)
    precision = calculatePrecision(TP, FP)
    f1 = calculateF1Score(precision, recall)

    return {
        'Accuracy': round(accuracy, 4),
        'Recall': round(recall, 4),
        'Precision': round(precision, 4),
        'F1': round(f1, 4)
    }

In [ ]:
def count_true_positives(gt_df, pred_df, column='Allocation', join_on='Study', positive_value='Random'):
    merged = gt_df[[join_on, column]].merge(pred_df[[join_on, column]], on=join_on, suffixes=('_gt', '_pred'))
    tp = ((merged[f"{column}_gt"] == positive_value) &
          (merged[f"{column}_pred"] == positive_value)).sum()
    return tp

def count_false_positives(gt_df, pred_df, column='Allocation', join_on='Study', positive_value='Random'):
    merged = gt_df[[join_on, column]].merge(pred_df[[join_on, column]], on=join_on, suffixes=('_gt', '_pred'))
    fp = ((merged[f"{column}_gt"] != positive_value) &
          (merged[f"{column}_pred"] == positive_value)).sum()
    return fp

def count_false_negatives(gt_df, pred_df, column= 'Allocation', join_on='Study', positive_value='Random'):
    merged = gt_df[[join_on, column]].merge(pred_df[[join_on, column]], on=join_on, suffixes=('_gt', '_pred'))
    fn = ((merged[f"{column}_gt"] == positive_value) &
          (merged[f"{column}_pred"] != positive_value)).sum()
    return fn

def count_true_negatives(gt_df, pred_df, column='Allocation', join_on='Study', positive_value='Random'):
    merged = gt_df[[join_on, column]].merge(pred_df[[join_on, column]], on=join_on, suffixes=('_gt', '_pred'))
    tn = ((merged[f"{column}_gt"] != positive_value) &
          (merged[f"{column}_pred"] != positive_value)).sum()
    return tn

In [ ]:
def binary_metrics(ground_truth_df, extracted_df, column='Allocation', join_on='Study', positive_value='Random'):
  tp = count_true_positives(ground_truth_df, extracted_df, column=column, join_on=join_on, positive_value=positive_value)
  fp = count_false_positives(ground_truth_df, extracted_df, column=column, join_on=join_on, positive_value=positive_value)
  fn = count_false_negatives(ground_truth_df, extracted_df, column=column, join_on=join_on, positive_value=positive_value)
  tn = count_true_negatives(ground_truth_df, extracted_df, column=column, join_on=join_on, positive_value=positive_value)

  # print(f'\nHere are pre-metrics for Allocation:')
  # print(f'TP: {tp}')
  # print(f'FP: {fp}')
  # print(f'FN: {fn}')
  # print(f'TN: {tn}\n\n')


  metrics = calculateMetrics(tp, fp, fn, tn)
  # print(f'Here are metrics for {column} column:')
  # for name, val in metrics.items():
  #   print(f"{name:10s}: {float(val):.2f}")
  return metrics

In [ ]:
def multiclass_metrics(ground_truth_df,
                       extracted_df,
                       column='Experimenter', join_on='Study',
                       values=['Teacher', 'Parent', 'Researcher', 'Combined - Teacher and Parent']):
  all_metrics = {}
  supports = {}
  # print(f'\nHere are pre-metrics for Experimenter:')
  for value in values:
    tp = count_true_positives(ground_truth_df, extracted_df, column=column, join_on=join_on, positive_value=value)
    fp = count_false_positives(ground_truth_df, extracted_df, column=column, join_on=join_on, positive_value=value)
    fn = count_false_negatives(ground_truth_df, extracted_df, column=column, join_on=join_on, positive_value=value)
    tn = count_true_negatives(ground_truth_df, extracted_df, column=column, join_on=join_on, positive_value=value)

    # print(f'\nHere are pre-metrics for {value}:')
    # print(f'TP: {tp}')
    # print(f'FP: {fp}')
    # print(f'FN: {fn}')
    # print(f'TN: {tn}\n\n')

    metrics = calculateMetrics(tp, fp, fn, tn)
    support = tp + fn  # number of true instances of this class

    # print(f'Metrics for {value}:')
    # for name, val in metrics.items():
    #     print(f"{name:10s}: {float(val):.2f}")
    # print('--------------------------------------')

  ##########################

    all_metrics[value] = metrics
    supports[value] = support


  total_support = sum(supports.values())
  weighted_metrics = {}
  for metric in ['Accuracy', 'Recall', 'Precision', 'F1']:
      weighted_metrics[metric] = (
          sum(all_metrics[cls][metric] * supports[cls] for cls in values)
          / total_support
      )

  # print(f"Weighted average metrics for '{column}':")
  # for name, val in weighted_metrics.items():
  #     print(f"{name:10s}: {val:.2f}")

  return all_metrics, weighted_metrics

  ##########################

  # avg_metrics = {
  #     'Accuracy': np.mean([m['Accuracy'] for m in all_metrics.values()]),
  #     'Recall': np.mean([m['Recall'] for m in all_metrics.values()]),
  #     'Precision': np.mean([m['Precision'] for m in all_metrics.values()]),
  #     'F1': np.mean([m['F1'] for m in all_metrics.values()])
  # }
  # print(f'Here are average metrics for {column} column:')
  # for name, val in avg_metrics.items():
  #     print(f"{name:10s}: {float(val):.2f}")
  # return metrics, avg_metrics

In [ ]:
# from sklearn.metrics import classification_report

# # Parameters
# num_samples = 20
# num_classes = 4
# seed = 42
# np.random.seed(seed)

# # Generate sample IDs
# ids = np.arange(1, num_samples + 1)

# # Generate ground-truth labels (even distribution)
# gt = np.repeat(np.arange(num_classes), num_samples // num_classes)
# remainder = num_samples - len(gt)
# gt = np.concatenate([gt, np.arange(remainder)])
# np.random.shuffle(gt)

# # Generate predictions: start with correct labels, then flip a portion
# pred = gt.copy()
# n_flip = int(0.2 * num_samples)  # flip 20% of samples
# flip_idx = np.random.choice(num_samples, n_flip, replace=False)
# for idx in flip_idx:
#     wrong_classes = list(set(range(num_classes)) - {gt[idx]})
#     pred[idx] = np.random.choice(wrong_classes)

# print(gt)
# print(pred)

# metrics, mult_metrics = multiclass_metrics(pd.DataFrame({'id': ids, 'label': gt}),
#                                                  pd.DataFrame({'id': ids, 'label': pred}),
#                                                  column='label', join_on='id', values=[0, 1, 2, 3])

#Prompting

In [ ]:
query_extraction ="""
You you are provided with examples of pdfs and csv tables corresponding to them.
A new pdf is attached.
Extract the following information **only** from this document:
### Allocation
Were the students allocated randomly to the experimental group? Random /  Non-random
{Random, Non-random}
### Experimenter
Who delivered the experimental intervention to the students/pupils? \
If the teacher or the parent was trained by the researcher,\
but the intervention was actually delivered by the teacher or the parent,\
then the researcher is not the experimenter.
{teacher, parent, researcher, combined:teacher and parent}


Return the output as a **stringified JSON array**, with no extra text:
        [
            {{
            "Allocation": "<answer>",
            "Experimenter": "<answer>"
            }}
        ]

Do **not** include any extra text or columns—return csv with the specified columns: "Allocation", "Experimenter", and "Study".
"""

In [ ]:
class AnswerItem(BaseModel):
  file: str
  row: int

In [ ]:
truth_df = pd.read_excel('/content/table/data.xlsx')
truth_df = truth_df[['Study_ID', 'Study', 'Allocation', 'Experimenter']]
truth_df = truth_df.drop_duplicates(subset=['Study_ID', 'Study']).reset_index(drop=True)

In [ ]:
from collections import defaultdict

def create_bins(matched_pairs):
  experimenter_bins = defaultdict(list)

  for file_path, group_df in matched_pairs:
    if not group_df.empty and 'Experimenter' in group_df.columns:
      experimenter_value = group_df['Experimenter'].iloc[0]
      experimenter_bins[experimenter_value].append((file_path, group_df))
    else:
      # Handle cases where the DataFrame is empty or 'Experimenter' column is missing
      experimenter_bins['Unknown'].append((file_path, group_df))
  return experimenter_bins


print("Bins created based on Experimenter:")
experimenter_bins = create_bins(matched_pairs)
for experimenter, pairs in experimenter_bins.items():
  print(f"- {experimenter}: {len(pairs)} samples")


Bins created based on Experimenter:
- Parent: 5 samples
- Researcher: 8 samples
- Teacher: 9 samples
- nan: 1 samples


In [ ]:
def prompt_loop(train, test, name, llm):
  extracted_all = pd.DataFrame()

  output_dir = f'/content/extracted_{name}'
  !rm -rf "{output_dir}"
  os.makedirs(output_dir, exist_ok=True)
  print(f"Directory '{output_dir}' created successfully.")

  c0 = time.process_time()

  for idx, (test_pdf_path, test_df) in enumerate(test):
      test_file = client.files.create(file=open(test_pdf_path, "rb"), purpose="assistants")
      ground_truth_messages = []

      for file_path, group_df in train:
          file = client.files.create(file=open(file_path, "rb"), purpose="assistants")

          message_content = (
              f"Here is a document ({os.path.basename(file_path)}) "
              f"and the data extracted from it:\n\n{group_df}"
          )

          ground_truth_messages.append({
              "role": "user",
              "content": message_content,
              "attachments": [{"file_id": file.id, "tools": [{"type": "file_search"}]}]
          })

      messages = [{
          "role": "user",
          "content": query_extraction,
          "attachments": [{"file_id": test_file.id, "tools": [{"type": "file_search"}]}]
      }] + ground_truth_messages

      result = llm.invoke(input=messages)
      assistant_response = result.content if hasattr(result, 'content') else str(result)

      # Add a delay to avoid rate limiting
      time.sleep(5)

      # print(f"--- Extraction for test sample {idx} ---")
      data = extract_json(assistant_response)
      df_extracted = json_to_dataframe(data)
      df_extracted['Study'] = os.path.basename(test_pdf_path)
      df_extracted['Study'] = df_extracted['Study'].apply(format)

      extracted_all = pd.concat([extracted_all, df_extracted], axis=0, ignore_index=True)

      df_extracted.to_csv(f"/content/extracted_{name}/{os.path.basename(test_pdf_path)}.csv", index=False)
      # output_csv = f"/content/extracted_{name}/{os.path.basename(test_pdf_path)}.csv"



  elapsed_cpu = time.process_time() - c0
  print(f"All samples processed in {elapsed_cpu:.2f} seconds.")

  # print("\n--- Metrics Check ---\n")
  bin_metrics = binary_metrics(truth_df, extracted_all)
  print('--------------------------------------')
  print('--------------------------------------\n')
  mult_metrics, mult_avg_metrics = multiclass_metrics(truth_df, extracted_all)
  print('--------------------------------------')
  print('--------------------------------------')
  return elapsed_cpu, bin_metrics, mult_metrics, mult_avg_metrics

In [ ]:
size = len(matched_pairs)
print(f"Total samples: {size}" )
split_range = range(4, 10)
shuffle_range = range(1, 4)

Total samples: 23


In [ ]:
from sklearn.model_selection import train_test_split

times = [[] for _ in shuffle_range]
bin_metrics_array = [[] for _ in shuffle_range]
mult_avg_metrics_array = [[] for _ in shuffle_range]

for n_examples in split_range:
  for i in shuffle_range:
    print("\n\n%%%%%%%%%%%%%%%%%%%%%%")
    print(f"Shuffle {i} & N Examples {n_examples}")
    print("%%%%%%%%%%%%%%%%%%%%%%")


    matched_df = pd.DataFrame(matched_pairs,
                              columns=['file_path', 'group_df'])
    matched_df['Study'] = matched_df['group_df'].apply(lambda x: format(x['Study'].iloc[0]))

    # Let's fill NaN with a placeholder like 'Unknown'
    matched_df['Experimenter'] = matched_df['group_df'].apply(lambda x: x['Experimenter'].iloc[0] if not x['Experimenter'].empty else 'Unknown').fillna('Unknown')
    matched_df['Allocation'] = matched_df['group_df'].apply(lambda x: x['Allocation'].iloc[0] if not x['Allocation'].empty else 'Unknown').fillna('Unknown')


    # Perform stratified split
    # Ensure all classes in the stratification columns have at least one sample
    # If any class combination has only one sample, train_test_split will raise an error.
    # Create a combined stratification column
    matched_df['Stratify_Combined'] = matched_df['Experimenter'] + '_' + matched_df['Allocation']
    stratify_col = 'Stratify_Combined'
    combined_counts = matched_df[stratify_col].value_counts()


    # Check if stratification is possible
    can_stratify = (combined_counts >= 2).all()

    if can_stratify:
        print(f"Performing stratified split with train size {n_examples} based on Experimenter and Allocation")
        train_indices, test_indices = train_test_split(
            matched_df.index,
            train_size=n_examples,
            stratify=matched_df[stratify_col],
            random_state=i
        )
    else:
        print("Warning: Cannot perform stratified split based on both Experimenter and Allocation due to classes with less than 2 samples in combined categories. Performing stratified split based on Experimenter only.")
        # Fallback to stratifying only on Experimenter if combined stratification is not possible
        experimenter_counts = matched_df['Experimenter'].value_counts()
        if (experimenter_counts >= 2).all():
             train_indices, test_indices = train_test_split(
                matched_df.index,
                train_size=n_examples, # Use train_size instead of test_size
                stratify=matched_df['Experimenter'],
                random_state=i # Use shuffle index as random_state
            )
        else:
             print("Warning: Cannot perform stratified split based on Experimenter either. Performing non-stratified split.")
             train_indices, test_indices = train_test_split(
                matched_df.index,
                train_size=n_examples, # Use train_size instead of test_size
                random_state=i # Use shuffle index as random_state
            )


    train_pairs_selected = [matched_pairs[idx] for idx in train_indices]
    test_pairs_selected = [matched_pairs[idx] for idx in test_indices]

    print(f"Created a training set with {len(train_pairs_selected)} samples.")
    print(f"Created a test set with {len(test_pairs_selected)} samples.")


    try:
        total_elapsed, bin_metrics, mult_metrics, mult_avg_metrics \
        = prompt_loop(train_pairs_selected, test_pairs_selected, f"examples_{n_examples}_shuffle_{i}", llm)
        # print("\n\n\n")

        times[i - 1].append(total_elapsed) # Adjust index to be 0-based
        bin_metrics_array[i - 1].append(bin_metrics) # Adjust index to be 0-based
        mult_avg_metrics_array[i - 1].append(mult_avg_metrics) # Adjust index to be 0-based
    except ValueError as e:
        print(f"Error processing examples_{n_examples}_shuffle_{i}: {e}")
        print("Skipping this run.")
        # Optionally append None or a placeholder to times, bin_metrics_array, etc.
        # to keep the structure consistent, if needed for later processing.
        # For now, we just skip appending, which means these lists might have
        # different lengths if some runs fail. Be mindful of this in plotting.



%%%%%%%%%%%%%%%%%%%%%%
Shuffle 1 & N Examples 4
%%%%%%%%%%%%%%%%%%%%%%
Created a training set with 4 samples.
Created a test set with 19 samples.
Directory '/content/extracted_examples_4_shuffle_1' created successfully.


All samples processed in 1.83 seconds.
--------------------------------------
--------------------------------------

--------------------------------------
--------------------------------------


%%%%%%%%%%%%%%%%%%%%%%
Shuffle 2 & N Examples 4
%%%%%%%%%%%%%%%%%%%%%%
Created a training set with 4 samples.
Created a test set with 19 samples.
Directory '/content/extracted_examples_4_shuffle_2' created successfully.


All samples processed in 1.71 seconds.
--------------------------------------
--------------------------------------

--------------------------------------
--------------------------------------


%%%%%%%%%%%%%%%%%%%%%%
Shuffle 3 & N Examples 4
%%%%%%%%%%%%%%%%%%%%%%
Created a training set with 4 samples.
Created a test set with 19 samples.
Directory '/content/extracted_examples_4_shuffle_3' created successfully.


All samples processed in 1.58 seconds.
--------------------------------------
--------------------------------------

--------------------------------------
--------------------------------------


%%%%%%%%%%%%%%%%%%%%%%
Shuffle 1 & N Examples 5
%%%%%%%%%%%%%%%%%%%%%%
Created a training set with 5 samples.
Created a test set with 18 samples.
Directory '/content/extracted_examples_5_shuffle_1' created successfully.


All samples processed in 1.89 seconds.
--------------------------------------
--------------------------------------

--------------------------------------
--------------------------------------


%%%%%%%%%%%%%%%%%%%%%%
Shuffle 2 & N Examples 5
%%%%%%%%%%%%%%%%%%%%%%
Created a training set with 5 samples.
Created a test set with 18 samples.
Directory '/content/extracted_examples_5_shuffle_2' created successfully.


...

Error processing examples_8_shuffle_2: Failed to parse JSON: Expecting property name enclosed in double quotes: line 2 column 4 (char 5)
Extracted string: [
  {{
    "Allocation": "Random",
    "Experimenter": "Teacher"
  }},
  {{
    "Allocation": "Random",
    "Experimenter": "Researcher"
  }},
  {{
    "Allocation": "Random",
    "Experimenter": "Teacher"
  }},
  {{
    "Allocation": "Random",
    "Experimenter": "NaN"
  }},
  {{
    "Allocation": "Random",
    "Experimenter": "Researcher"
  }},
  {{
    "Allocation": "Random",
    "Experimenter": "Teacher"
  }},
  {{
    "Allocation": "Random",
    "Experimenter": "Parent"
  }},
  {{
    "Allocation": "Non-random",
    "Experimenter": "Teacher"
  }}
]
Skipping this run.


%%%%%%%%%%%%%%%%%%%%%%
Shuffle 3 & N Examples 8
%%%%%%%%%%%%%%%%%%%%%%
Created a training set with 8 samples.
Created a test set with 15 samples.
Directory '/content/extracted_examples_8_shuffle_3' created successfully.


All samples processed in 2.07 seconds.
--------------------------------------
--------------------------------------

--------------------------------------
--------------------------------------


%%%%%%%%%%%%%%%%%%%%%%
Shuffle 1 & N Examples 9
%%%%%%%%%%%%%%%%%%%%%%
Created a training set with 9 samples.
Created a test set with 14 samples.
Directory '/content/extracted_examples_9_shuffle_1' created successfully.


All samples processed in 1.88 seconds.
--------------------------------------
--------------------------------------

--------------------------------------
--------------------------------------


%%%%%%%%%%%%%%%%%%%%%%
Shuffle 2 & N Examples 9
%%%%%%%%%%%%%%%%%%%%%%
Created a training set with 9 samples.
Created a test set with 14 samples.
Directory '/content/extracted_examples_9_shuffle_2' created successfully.


Error processing examples_9_shuffle_2: Failed to parse JSON: Expecting property name enclosed in double quotes: line 2 column 6 (char 7)
Extracted string: [
    {{
    "Allocation": "Random",
    "Experimenter": "Researcher"
    }},
    {{
    "Allocation": "Random",
    "Experimenter": "Teacher"
    }},
    {{
    "Allocation": "Random",
    "Experimenter": "Researcher"
    }},
    {{
    "Allocation": "Random",
    "Experimenter": "Teacher"
    }},
    {{
    "Allocation": "Random",
    "Experimenter": "Teacher"
    }},
    {{
    "Allocation": "Random",
    "Experimenter": "Parent"
    }},
    {{
    "Allocation": "Non-random",
    "Experimenter": "Teacher"
    }}
]
Skipping this run.


%%%%%%%%%%%%%%%%%%%%%%
Shuffle 3 & N Examples 9
%%%%%%%%%%%%%%%%%%%%%%
Created a training set with 9 samples.
Created a test set with 14 samples.
Directory '/content/extracted_examples_9_shuffle_3' created successfully.


All samples processed in 1.87 seconds.
--------------------------------------
--------------------------------------

--------------------------------------
--------------------------------------


In [ ]:
# from google.colab import files
# import os
# import shutil

# for split in split_range:
#   for shuffle in shuffle_range:
#     folder_to_download = f'/content/extracted_examples_{split}_shuffle_{shuffle}'

#     # Create a zip archive of the folder
#     zip_path = f'{folder_to_download.split("/")[-1]}.zip'
#     shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_to_download)

#     # Download the zip file
#     files.download(zip_path)

#Plotting

In [ ]:
def find_best_shuffle(mult_avg_metrics_array, shuffle_range):
    """
    Finds the shuffle with the best average multiclass F1 score across split sizes.

    Args:
        mult_avg_metrics_array (list of list of dict): List of lists containing
            multiclass average metrics dictionaries for each shuffle and split size.
        shuffle_range (range): The range of shuffle indices (1-based).

    Returns:
        tuple: A tuple containing the index of the best shuffle (1-based) and its
               average F1 score, or (None, None) if no metrics are available.
    """
    best_f1 = -1
    best_shuffle_index = None

    for i in shuffle_range:
        shuffle_idx = i - 1  # Adjust to 0-based index
        shuffle_metrics = mult_avg_metrics_array[shuffle_idx]

        if shuffle_metrics:
            # Calculate the average F1 score for the current shuffle across all split sizes
            avg_f1 = sum(m.get('F1', 0) for m in shuffle_metrics) / len(shuffle_metrics)

            if avg_f1 > best_f1:
                best_f1 = avg_f1
                best_shuffle_index = i  # Store 1-based index

    return best_shuffle_index, best_f1

# Find the best shuffle based on average multiclass F1 score
best_shuffle, best_f1_score = find_best_shuffle(mult_avg_metrics_array, shuffle_range)

if best_shuffle is not None:
    print(f"The best shuffle based on average multiclass F1 score is Shuffle {best_shuffle} with an average F1 of {best_f1_score:.4f}")
else:
    print("Could not determine the best shuffle.")

The best shuffle based on average multiclass F1 score is Shuffle 2 with an average F1 of 0.3563


In [ ]:
import matplotlib.pyplot as plt

# Assuming shuffle_range is defined and corresponds to the shuffles performed (e.g., range(1, 10))
# Assuming split_range is defined and corresponds to the splits performed (e.g., range(4, 5))
# Assuming times, bin_metrics_array, and mult_avg_metrics_array are populated as before.

def plot_results(split_sizes, times_data, bin_metrics_data, mult_avg_metrics_data, shuffle_index=None):
    """
    Plot total elapsed time and evaluation metrics across split sizes.

    Args:
        split_sizes (list of int): The numbers of examples used for the plots.
        times_data (list of list of float): List of lists containing elapsed times
            for each shuffle and split size.
        bin_metrics_data (list of list of dict): List of lists containing binary metrics
            dictionaries for each shuffle and split size.
        mult_avg_metrics_data (list of list of dict): List of lists containing multiclass
            average metrics dictionaries for each shuffle and split size.
        shuffle_index (int, optional): The 1-based index of the shuffle to plot.
            If None, plots for all shuffles. Defaults to None.
    """
    if shuffle_index is not None:
        # Plot results for a specific shuffle
        shuffle_idx = shuffle_index - 1 # Adjust to 0-based index
        if shuffle_idx < len(times_data) and len(times_data[shuffle_idx]) == len(split_sizes):
            print(f"Plotting results for Shuffle: {shuffle_index}")

            # Plot total elapsed time vs number of examples
            plt.figure()
            plt.plot(split_sizes, times_data[shuffle_idx], marker='o')
            plt.xlabel('Number of Examples')
            plt.ylabel('Total Elapsed CPU Time (s)')
            plt.title(f'Elapsed CPU Time vs Number of Examples (Shuffle = {shuffle_index})')
            plt.grid(True)
            plt.xticks(split_sizes)
            plt.show()

            # Extract metric names
            metric_names = ['Accuracy', 'Recall', 'Precision', 'F1']

            # Plot binary metrics
            plt.figure()
            for name in metric_names:
                values = [m[name] for m in bin_metrics_data[shuffle_idx]]
                plt.plot(split_sizes, values, marker='o', label=name)
            plt.xlabel('Number of Examples')
            plt.ylabel('Binary Metric Value')
            plt.title(f'Binary Metrics vs Number of Examples (Shuffle = {shuffle_index})')
            plt.legend()
            plt.grid(True)
            plt.xticks(split_sizes)
            plt.show()

            # Plot multiclass average metrics
            plt.figure()
            for name in metric_names:
                values = [m[name] for m in mult_avg_metrics_data[shuffle_idx]]
                plt.plot(split_sizes, values, marker='o', label=name)
            plt.xlabel('Number of Examples')
            plt.ylabel('Multiclass Average Metric')
            plt.title(f'Multiclass Avg Metrics vs Number of Examples (Shuffle = {shuffle_index})')
            plt.legend()
            plt.grid(True)
            plt.xticks(split_sizes)
            plt.show()
        else:
            print(f"Data for Shuffle {shuffle_index} is incomplete or not available for plotting across all split sizes.")
    else:
        # Plot results for all shuffles (original behavior)
        for i in shuffle_range:
            shuffle_idx = i - 1 # Adjust to 0-based index
            print(f"Plotting results for Shuffle: {i}")

            current_shuffle_times = []
            current_shuffle_bin_metrics = []
            current_shuffle_mult_avg_metrics = []
            completed_split_sizes = []

            for split_idx, split_size in enumerate(split_range):
                 if shuffle_idx < len(times_data) and split_idx < len(times_data[shuffle_idx]):
                    current_shuffle_times.append(times_data[shuffle_idx][split_idx])
                    current_shuffle_bin_metrics.append(bin_metrics_data[shuffle_idx][split_idx])
                    current_shuffle_mult_avg_metrics.append(mult_avg_metrics_data[shuffle_idx][split_idx])
                    completed_split_sizes.append(split_size)


            plot_results(completed_split_sizes, [current_shuffle_times], [current_shuffle_bin_metrics], [current_shuffle_mult_avg_metrics], shuffle_index=i) # Recursive call for single shuffle plotting


# Call the plotting function with the best shuffle index
plot_results(list(split_range), times, bin_metrics_array, mult_avg_metrics_array, shuffle_index=best_shuffle)

Data for Shuffle 2 is incomplete or not available for plotting across all split sizes.
